In [1]:
import json
# EXPERIMENTAL
# Work in progress (do not use)

# find TC genesis from models based on work similar to FSU's methodology

# only focused on 1. so far:
# 1. https://journals.ametsoc.org/view/journals/wefo/28/6/waf-d-13-00008_1.xml
# 2. https://journals.ametsoc.org/view/journals/wefo/31/3/waf-d-15-0157_1.xml?tab_body=fulltext-display
# 3. https://journals.ametsoc.org/view/journals/wefo/32/1/waf-d-16-0072_1.xml

# partial TODO
#    extend/generalize to other models
#        all other models are parameter split gribs
#             including another source for NAVGEM which has 850 relative vorticity
#        refactor code for lat/lons
#             check don't need separate lat lons for each variable (same resolution or array shape))
#    get max 10m speed if available
#        87:10 metre U wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 96 hrs:from 202310190600
#        88:10 metre V wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 96 hrs:from 202310190600
#    save TC output to JSON
#    extend to handle multiple timesteps/bufrs
#        individually a single criteria being met is a disturbance,
#             start of 24 continuous hours of meeting criteriais a tc
#    automatically download model files (UKMET seems infeasible as it costs much money)

# thresholds for disturbances (reversed from text output from FSU)
disturbance_thresholds_path = 'disturbance_thresholds.json'

# shape file for placing lat,lon in basins which we are classifying
# each has an attribute called 'basin_name', with CPAC & EPAC combined as EPAC
# basins are NATL, EPAC, WPAC, IO, SH
shape_file = 'shapes/basins.shp'

# Test the function with your GRIB file
# NAVGEM 96h
# uses regular_ll

#grib_file = "navgem_2023101906f096.grib2"
# cgi combined all relevant?
# the new was with cgi but it gets TOO much information (it gets 11 parameters rather than 9: u/v wind for 200 we don't need)
##### grib_file = "/home/db/metview/JRPdata/gfs/gfs.t00z.pgrb2.0p25.f000_new"
#   use the index file
#grib_file = "/home/db/metview/JRPdata/navgem2/US058GMET-GR1mdl.0018_0056_09600F0OF2023101906_0100_008500-000000geop_ht"
#grib_file = "/home/db/metview/JRPdata/ukmet/isbl_geopotential-height_85000.0_2023102112_12.grib"

# only tested the unified grib with the NAV model (as the NCEP NAV grib data is unified)
model_name = 'NAV'

In [2]:
import pygrib
import re
import math
from metpy.units import units
import metpy.calc as mpcalc
import numpy as np
import pyproj
import geopandas as gpd
from shapely.geometry import Point

# may need to modify lzma.py in python folder to get this to work (pip install backports-lzma)
# this is for metpy
try:
    import lzma
except ImportError:
    import backports.lzma as lzma

gdf = gpd.read_file(shape_file)

disturbance_criteria_names_map = {
    'WS': 'vmax925',
    'THKN': 'gp250_850_thickness',
    'RV': 'rv850max'
}

# shape file attribute names to threshold names
shape_basin_names_to_threshold_names_map = {
    'NATL': 'AL',
    'WPAC': 'WP',
    'IO': 'IO',
    'SH': 'SH',
    'EPAC': 'EP'
}

with open(disturbance_thresholds_path, 'r') as f:
    disturbance_thresholds = json.loads(f.read())

In [3]:
# todo: the calculations using neighborhoods don't wrap around at lons (and lats) and they should
#  for instance, the unsigned lats array starts at 0 degrees and ends at 360

def list_available_parameters(grib_file):
    try:
        # Open the GRIB file
        grbs = pygrib.open(grib_file)

        # Initialize a list to store parameter information
        parameter_info = []

        # Iterate through the GRIB messages and extract parameter information
        for grb in grbs:
            parameter_name = grb.name
            parameter_unit = grb.units
            level_type = grb.levelType
            level = grb.level
            print(grb)
            parameter_info.append({
                "Parameter Name": parameter_name,
                "Unit": parameter_unit,
                "Level Type": level_type,
                "Level": level
            })

        # Close the GRIB file
        grbs.close()

        # Print the information for parameters
        for info in parameter_info:
            print("Parameter Name:", info["Parameter Name"])
            print("Unit:", info["Unit"])
            print("Level Type:", info["Level Type"])
            print("Level:", info["Level"])
            print("\n")

    except Exception as e:
        print(f"Error: {e}")

def load_and_extract_unified_grib_data(grib_file):
    try:
        # Open the GRIB file
        grbs = pygrib.open(grib_file)

        # Initialize variables for relevant parameters
        mslp = None
        u_wind_925 = None
        v_wind_925 = None
        u_wind_850 = None
        v_wind_850 = None
        geopotential_250 = None
        geopotential_850 = None
        
        mslp_units = None
        wind_925_units = None
        
        mslp_lats = None
        mslp_lons = None
        u_wind_925_lats = None
        u_wind_925_lons = None
        v_wind_925_lats = None
        v_wind_925_lons = None
        u_wind_850_lats = None
        u_wind_850_lons = None
        v_wind_850_lats = None
        v_wind_850_lons = None
        geopotential_250_lats = None
        geopotential_250_lons = None
        geopotential_850_lats = None
        geopotential_850_lons = None
        
        grid_resolution = {}
        
        # Extract relevant parameters (modify the parameter names and levels accordingly)
        for grb in grbs:
            if grb.name == 'Pressure reduced to MSL' and grb.level == 0:
                mslp = grb.values
                mslp_units = grb.units
                mslp_lats, mslp_lons = grb.latlons()
                grid_resolution['mslp'] = grb['iDirectionIncrementInDegrees']
            elif grb.name == 'U component of wind' and grb.level == 925:
                u_wind_925 = grb.values
                wind_925_units = grb.units
                u_wind_925_lats, u_wind_925_lons = grb.latlons()
                grid_resolution['uwind925'] = grb['iDirectionIncrementInDegrees']
            elif grb.name == 'V component of wind' and grb.level == 925:
                v_wind_925 = grb.values
                v_wind_925_lats, v_wind_925_lons = grb.latlons()
                grid_resolution['vwind925'] = grb['iDirectionIncrementInDegrees']
            elif grb.name == 'U component of wind' and grb.level == 850:
                u_wind_850 = grb.values
                wind_850_units = grb.units
                u_wind_850_lats, u_wind_850_lons = grb.latlons()
                grid_resolution['uwind850'] = grb['iDirectionIncrementInDegrees']
            elif grb.name == 'V component of wind' and grb.level == 850:
                v_wind_850 = grb.values
                v_wind_850_lats, v_wind_850_lons = grb.latlons()
                grid_resolution['vwind850'] = grb['iDirectionIncrementInDegrees']
            elif grb.name == 'Geopotential Height' and grb.level == 250:
                geopotential_250 = grb.values
                geopotential_250_lats, geopotential_250_lons = grb.latlons()
                grid_resolution['gh250'] = grb['iDirectionIncrementInDegrees']
            elif grb.name == 'Geopotential Height' and grb.level == 850:
                geopotential_850 = grb.values
                geopotential_850_lats, geopotential_850_lons = grb.latlons()
                grid_resolution['gh850'] = grb['iDirectionIncrementInDegrees']

        # Close the GRIB file
        grbs.close()


        # Check if units are not in hPa and convert if necessary
        if mslp_units != "hPa":
            print(f"Converting MSLP units from {mslp_units} to hPa")
            if mslp_units == "Pa":
                # Convert from Pa to hPa
                mslp *= 0.01
                mslp_units = "hPa"
            else:
                print("Warning: Units of MSLP are not in Pa or hPa. Please verify the units for accurate results.")

        # Check if units are not in m/s for 925 hPa wind components and convert if necessary
        if not (re.search(r"(m/s|m s\*\*-1)", wind_925_units)):
            print(f"Converting 925 hPa wind components units from {wind_925_units} to m/s")
            if re.search(r"knots|knot", wind_925_units, re.I):
                # Convert from knots to m/s (1 knot ≈ 0.514444 m/s)
                u_wind_925 *= 0.514444
                v_wind_925 *= 0.514444
                wind_925_units = "m/s"
            else:
                print("Warning: Units of 925 hPa wind components are not in knots, m/s, or m s**-1. Please verify the units for accurate results.")
                
        # Check if units are not in m/s for 925 hPa wind components and convert if necessary
        if not (re.search(r"(m/s|m s\*\*-1)", wind_850_units)):
            print(f"Converting 850 hPa wind components units from {wind_850_units} to m/s")
            if re.search(r"knots|knot", wind_925_units, re.I):
                # Convert from knots to m/s (1 knot ≈ 0.514444 m/s)
                u_wind_850 *= 0.514444
                v_wind_850 *= 0.514444
                wind_850_units = "m/s"
            else:
                print("Warning: Units of 925 hPa wind components are not in knots, m/s, or m s**-1. Please verify the units for accurate results.")
        
        return grid_resolution, mslp_units, mslp, mslp_lats, mslp_lons, wind_925_units, u_wind_925, u_wind_925_lats, u_wind_925_lons, v_wind_925, v_wind_925_lats, v_wind_925_lons, wind_850_units, u_wind_850, u_wind_850_lats, u_wind_850_lons, v_wind_850, v_wind_850_lats, v_wind_850_lons, geopotential_250, geopotential_250_lats, geopotential_250_lons, geopotential_850, geopotential_850_lats, geopotential_850_lons
    except Exception as e:
        print(f"Error: {e}")
        return None, None, None, None, None

def convert_to_signed_lon(lon):
    # Convert longitudes from 0-360 range to -180 to +180 range
    return (lon + 180) % 360 - 180


def calculate_neighborhood_size(degree_radius, grid_resolution):
    # Calculate the radius in grid points
    radius_in_grid_points = int(degree_radius / grid_resolution)

    # Calculate the neighborhood size
    neighborhood_size = 2 * radius_in_grid_points + 1  # Assuming a square neighborhood

    return neighborhood_size

def array_indices_to_lat_lon(x, y, mslp_lats, mslp_lons):
    lat = mslp_lats[x, y]
    lon = mslp_lons[x, y]
    signed_lon = convert_to_signed_lon(lon)
    return lat, signed_lon

# print candidates
def print_candidates(mslp_minima_list, mslp_lats = None, mslp_lons = None, meet_all_disturbance_thresholds = False):
    n = 0
    for candidate in mslp_minima_list:
        if meet_all_disturbance_thresholds:
            if not candidate['criteria']['all']:
                continue
                

        basin_str = ""
        if 'basin' in candidate:
            basin = candidate['basin']
            basin_str += f'{basin} Basin, '
        
        n += 1
        mslp_value = candidate["mslp_value"]
        
        if mslp_lats is not None and mslp_lons is not None:
            x = candidate["x_value"]
            y = candidate["y_value"]
            lat, lon = array_indices_to_lat_lon(x, y, mslp_lats, mslp_lons)
        else:
            lat = candidate['lat']
            lon = candidate['lon']
            
        formatted_mslp = f"{mslp_value:.1f}".rjust(6, ' ')
        rv_str = ""
        if 'rv850max'in candidate:
            rv_str = ", 850 RV MAX (*10^-5 1/s): "
            rv_value = candidate['rv850max'] * np.power(10.0,5)
            formatted_rv = f"{rv_value:2.2f}".rjust(5, ' ')
            rv_str += formatted_rv
        
        thickness_str = ""
        if 'gp250_850_thickness' in candidate:
            thickness_str = ", 250-850 hPa Thickness (m): "
            thickness_value = candidate['gp250_850_thickness']
            formatted_thickness = f"{thickness_value:2.2f}".rjust(6, ' ')
            thickness_str += formatted_thickness
        
        vmax_str = ""
        if 'vmax925' in candidate:
            vmax_str = ", 925 hPa WS MAX (m/s): "
            vmax_value = candidate['vmax925']
            formatted_vmax = f"{vmax_value:3.2f}".rjust(6, ' ')
            vmax_str += formatted_vmax
        
        print(f"#{n: >2}, {basin_str}Latitude (deg): {lat: >6}, Longitude (deg): {lon: >6}, MSLP (hPa): {formatted_mslp}{rv_str}{thickness_str}{vmax_str}")

# this function is for checking find_mslp_minima_with_closed_isobars
def find_mslp_minima(mslp_data, minima_neighborhood_size=3):
    try:
        # Lists to store MSLP minima, latitudes, and longitudes
        candidates = []

        # Loop through each grid point
        for x in range(minima_neighborhood_size, mslp_data.shape[0] - minima_neighborhood_size):
            for y in range(minima_neighborhood_size, mslp_data.shape[1] - minima_neighborhood_size):
                mslp_value = mslp_data[x, y]  # MSLP value at the current point
                neighborhood = mslp_data[x - minima_neighborhood_size:x + minima_neighborhood_size + 1,
                                         y - minima_neighborhood_size:y + minima_neighborhood_size + 1]

                # Check if the MSLP value is the minimum within the neighborhood
                if mslp_value == neighborhood.min():
                    candidate = {
                        "mslp_value": mslp_value,
                        "x_value": x,
                        "y_value": y
                    }
                    candidates.append(candidate)


        return candidates
    except Exception as e:
        print(f"Error: {e}")
        return [], [], []

def find_mslp_minima_with_closed_isobars(mslp_data, grid_resolution, minima_neighborhood_size=3, isobar_threshold=2.0, isobar_search_radius_degrees = 5):
    isobar_neighborhood_size = calculate_neighborhood_size(isobar_search_radius_degrees, grid_resolution)
    
    def dfs(x, y):
        # Depth-first search to find a closed path
        nonlocal path_found
        visited[x][y] = True

        for dx in range(-1, 2):
            for dy in range(-1, 2):
                nx, ny = x + dx, y + dy

                if 0 <= nx < isobar_neighborhood_size * 2 + 1 and 0 <= ny < isobar_neighborhood_size * 2 + 1 and not visited[nx][ny]:
                    if neighborhood_modified[nx][ny] >= 0:
                        if nx == 0 or ny == 0 or nx == isobar_neighborhood_size * 2 or ny == isobar_neighborhood_size * 2:
                            path_found = True
                            return
                        dfs(nx, ny)

    try:
        # List to store MSLP minima as dictionaries
        mslp_minima_list = []

        # Create a list of candidates for MSLP minima
        candidates = []

        # Loop through each grid point
        for x in range(minima_neighborhood_size, mslp_data.shape[0] - minima_neighborhood_size):
            for y in range(minima_neighborhood_size, mslp_data.shape[1] - minima_neighborhood_size):
                mslp_value = mslp_data[x, y]  # MSLP value at the current point
                neighborhood = mslp_data[x - minima_neighborhood_size:x + minima_neighborhood_size + 1,
                                         y - minima_neighborhood_size:y + minima_neighborhood_size + 1]

                # Check if the MSLP value is the minimum within the neighborhood
                if mslp_value == neighborhood.min():
                    candidates.append((x, y, mslp_value))

        # Loop through the candidates to find isobars
        for x, y, minima_value in candidates:
            # Create a modified neighborhood for isobar calculation
            x_min = max(0, x - isobar_neighborhood_size)
            x_max = min(mslp_data.shape[0], x + isobar_neighborhood_size + 1)
            y_min = max(0, y - isobar_neighborhood_size)
            y_max = min(mslp_data.shape[1], y + isobar_neighborhood_size + 1)

            neighborhood_modified = mslp_data[x_min:x_max, y_min:y_max] - minima_value - isobar_threshold

            # Initialize visited array for DFS
            visited = np.zeros_like(neighborhood_modified, dtype=bool)

            # Flag to track if a closed path is found
            path_found = False

            # Start DFS from the center of the neighborhood
            dfs(x - x_min, y - y_min)

            if path_found:
                # Store MSLP minima data as a dictionary
                candidate = {
                    "mslp_value": minima_value,
                    "x_value": x,
                    "y_value": y
                }
                mslp_minima_list.append(candidate)

        return mslp_minima_list

    except Exception as e:
        print(f"Error: {e}")
        return []

# calculate vorticity (accounting for projection distortion) using metpy (uses finite differences method)
# returns vorticity using an ellipsoid based on WGS84
def calculate_vorticity(u_wind_850, v_wind_850, mslp_lats, mslp_lons):
    # Calculate grid spacing distances (these will be the dx and dy values used for calculating vorticity)
    ellipsoid = pyproj.Geod(ellps="WGS84")
    mslp_dx, mslp_dy = mpcalc.lat_lon_grid_deltas(mslp_lons * units.degrees, mslp_lats * units.degrees, geod=ellipsoid)

    # Convert wind components to units
    u_wind_850_with_units = units.Quantity(u_wind_850, 'm/s')
    v_wind_850_with_units = units.Quantity(v_wind_850, 'm/s')

    # Calculate relative vorticity (use WGS84)
    vort_850 = np.array(mpcalc.vorticity(u_wind_850_with_units, v_wind_850_with_units, dx=mslp_dx, dy=mslp_dy))

    return vort_850

# Function to find RV maximums in neighborhoods for a list of candidates
def find_rv_maximums_in_neighborhoods(mslp_minima_list, rv, grid_resolution, relative_vorticity_radius_degrees = 2):
    rv_neighborhood_size = calculate_neighborhood_size(relative_vorticity_radius_degrees, grid_resolution)
    updated_mslp_minima_list = []

    for candidate in mslp_minima_list:
        x, y, _ = candidate["x_value"], candidate["y_value"], candidate["mslp_value"]

        # Extract the neighborhood for the current candidate
        x_min = max(0, x - rv_neighborhood_size)
        x_max = min(rv.shape[0], x + rv_neighborhood_size + 1)
        y_min = max(0, y - rv_neighborhood_size)
        y_max = min(rv.shape[1], y + rv_neighborhood_size + 1)
        neighborhood = rv[x_min:x_max, y_min:y_max]

        # Find the maximum RV value within the neighborhood
        rv_max = neighborhood.max()

        # Update a copy of the candidate's dictionary with rv maximum value
        updated_candidate = candidate.copy()
        updated_candidate["rv850max"] = rv_max

        # Add the updated candidate to the list
        updated_mslp_minima_list.append(updated_candidate)

    return updated_mslp_minima_list
    
# calculate relative max thickness for 250hPa - 850hPa for a list of candidates
def find_gp_250_850_max_thickness(mslp_minima_list, geopotential_250, geopotential_850, grid_resolution, degrees_radius=2):
    # Calculate the neighborhood size based on degrees_radius and grid_resolution
    neighborhood_size = calculate_neighborhood_size(degrees_radius, grid_resolution)

    updated_mslp_minima_list = []
    
    # Get the shape of the geopotential arrays
    array_shape = geopotential_250.shape

    # Iterate over the list of candidates
    for candidate in mslp_minima_list:
        # Extract the x and y indices of the candidate
        x_index, y_index = candidate['x_value'], candidate['y_value']

        # Calculate the neighborhood bounds
        x_start = max(0, x_index - neighborhood_size)
        x_end = min(array_shape[0] - 1, x_index + neighborhood_size)
        y_start = max(0, y_index - neighborhood_size)
        y_end = min(array_shape[1] - 1, y_index + neighborhood_size)

        # Extract the neighborhoods for 250 hPa and 850 hPa
        neighborhood_250 = geopotential_250[x_start:x_end + 1, y_start:y_end + 1]
        neighborhood_850 = geopotential_850[x_start:x_end + 1, y_start:y_end + 1]

        # Calculate the 250–850-hPa thickness for each cell in the neighborhood
        thickness = neighborhood_250 - neighborhood_850

        # Find the maximum thickness value in the neighborhood
        max_thickness = thickness.max()

        # Update a copy of the candidate's dictionary with the maximum thickness value
        updated_candidate = candidate.copy()
        updated_candidate['gp250_850_thickness'] = max_thickness
        
        # Add the updated candidate to the list
        updated_mslp_minima_list.append(updated_candidate)

    return updated_mslp_minima_list

# find max wind at 925 hPa for a list of candidates
def find_max_wind_925(mslp_minima_list, u_wind_925, v_wind_925, grid_resolution, degrees_radius=5):
    # Calculate the neighborhood size based on radius_degrees and grid_resolution for wind
    neighborhood_size = calculate_neighborhood_size(degrees_radius, grid_resolution)

    updated_mslp_minima_list = []
    
    # Get the shape of the wind arrays
    array_shape = u_wind_925.shape

    # Iterate over the list of candidates
    for candidate in mslp_minima_list:
        # Extract the x and y indices of the candidate
        x_index = candidate['x_value']
        y_index = candidate['y_value']

        # Calculate the neighborhood bounds
        x_start = max(0, x_index - neighborhood_size)
        x_end = min(array_shape[0] - 1, x_index + neighborhood_size)
        y_start = max(0, y_index - neighborhood_size)
        y_end = min(array_shape[1] - 1, y_index + neighborhood_size)

        # Extract the neighborhoods for u-wind and v-wind at 925 hPa
        neighborhood_u_wind = u_wind_925[x_start:x_end + 1, y_start:y_end + 1]
        neighborhood_v_wind = v_wind_925[x_start:x_end + 1, y_start:y_end + 1]

        # Calculate wind speed in the neighborhood
        wind_speed = np.sqrt(neighborhood_u_wind ** 2 + neighborhood_v_wind ** 2)

        # Find the maximum wind speed value in the neighborhood
        max_wind_speed = wind_speed.max()

        # Update a copy of the candidate's dictionary with the maximum wind speed value
        updated_candidate = candidate.copy()
        updated_candidate['vmax925'] = max_wind_speed
        
        # Add the updated candidate to the list
        updated_mslp_minima_list.append(updated_candidate)

    return updated_mslp_minima_list

# add basin name, and also removes candidates that aren't in one of the basins
def add_basin_name(mslp_minima_list, mslp_lats, mslp_lons):
    updated_mslp_minima_list = []

    for candidate in mslp_minima_list:
        x = candidate["x_value"]
        y = candidate["y_value"]

        lat, lon = array_indices_to_lat_lon(x, y, mslp_lats, mslp_lons)
        basin_name = get_basin_name_from_lat_lon(lat, lon)
        
        # remove candidate if not in a basin we are considering
        if not basin_name:
            continue

        # Update a copy of the candidate's dictionary with rv maximum value
        updated_candidate = candidate.copy()
        updated_candidate['basin'] = basin_name
        
        updated_mslp_minima_list.append(updated_candidate)

    return updated_mslp_minima_list

# Function to calculate the booleans on disturbance thresholds are met or not for each criteria and for all
# Must calculate and update with the basin classification first
def calc_disturbance_threshold_booleans(mslp_minima_list, model_name):
    updated_mslp_minima_list = []

    for candidate in mslp_minima_list:
        # Update a copy of the candidate's dictionary with rv maximum value
        updated_candidate = candidate.copy()
        all_met = True
        
        basin_name = candidate["basin"]
        
        for criteria_abbrev, criteria_value in disturbance_thresholds[model_name][basin_name].items():
            criteria_name = disturbance_criteria_names_map[criteria_abbrev]
            if criteria_name == 'rv850max':
                # thresholds in the JSON for RV are scaled by 10^5
                criteria_value *= pow(10, -5)
            criteria_bool = False
            if updated_candidate[criteria_name] >= criteria_value:
                criteria_bool = True
            if 'criteria' not in updated_candidate:
                updated_candidate['criteria'] = {}
            updated_candidate['criteria'][criteria_name] = criteria_bool
            all_met = (all_met and criteria_bool)

        updated_candidate['criteria']['all'] = all_met
        # Add the updated candidate to the list
        updated_mslp_minima_list.append(updated_candidate)

    return updated_mslp_minima_list

# returns the basin name from lat, lon if in a basin that is covered by the thresholds, otherwise returns None
def get_basin_name_from_lat_lon(lat, lon):
    # Create a Point geometry for the latitude and longitude
    point = Point(lon, lat)

    # Check if the point is within any of the polygons
    result = gdf[gdf.geometry.covers(point)]
    if not result.empty:
        shape_basin_name = result['basin_name'].iloc[0]
        # this is used for thresholds so return the threshold name
        return shape_basin_names_to_threshold_names_map[shape_basin_name]
    else:
        return None

# return with lat and lon
# filter out candidates not meeting criteria
def get_disturbance_candidates_from_unified_grib(grib_file):
    grid_resolution, mslp_units, mslp, mslp_lats, mslp_lons, wind_925_units, u_wind_925, u_wind_925_lats, u_wind_925_lons, v_wind_925, v_wind_925_lats, v_wind_925_lons, wind_850_units, u_wind_850, u_wind_850_lats, u_wind_850_lons, v_wind_850, v_wind_850_lats, v_wind_850_lons, geopotential_250, geopotential_250_lats, geopotential_250_lons, geopotential_850, geopotential_850_lats, geopotential_850_lons = load_and_extract_unified_grib_data(grib_file)

    shapes = [x.shape for x in
        [
            mslp_lats, mslp_lons, u_wind_925_lats, u_wind_925_lons, v_wind_925_lats, v_wind_925_lons,
            u_wind_850_lats, u_wind_850_lons, v_wind_850_lats, v_wind_850_lons,
            geopotential_250_lats, geopotential_250_lons, geopotential_850_lats, geopotential_850_lons
        ] if x is not None]
    
    # check to make sure all the same shape
    if len(set(shapes)) != 1:
        # lats and lons different shapes!
        print(f"Error: getting disturbance candidates: lat,lons different shapes for: {grib_file}")
        return None
    
    # lats and lons are all the same
    lats = mslp_lats
    lons = mslp_lons
    
    mslp_minima_list = find_mslp_minima(mslp)

    # this will be the candidates restricted by MSLP closed isobars only
    mslp_minima_list_with_closed_isobars = find_mslp_minima_with_closed_isobars(mslp, grid_resolution['mslp'])

    # calculate relative vorticity for the entire data set first
    # this may issue warnings for divide by 0 for vorticity calculations
    rv_850 = calculate_vorticity(u_wind_850, v_wind_850, mslp_lats, mslp_lons)
    # calculate relative vorticity maximum for each candidate
    mslp_minima_list_with_closed_isobars = find_rv_maximums_in_neighborhoods(mslp_minima_list_with_closed_isobars, rv_850, grid_resolution['gh250'])

    # Find the maximum relative thickness (250 - 850 hPa) for each candidate
    mslp_minima_list_with_closed_isobars = find_gp_250_850_max_thickness(mslp_minima_list_with_closed_isobars, geopotential_250, geopotential_850, grid_resolution['gh250'])

    # find the maximum 925 hPa wind speed
    mslp_minima_list_with_closed_isobars = find_max_wind_925(mslp_minima_list_with_closed_isobars, u_wind_925, v_wind_925, grid_resolution['uwind925'])

    mslp_minima_list_in_basins = add_basin_name(mslp_minima_list_with_closed_isobars, lats, lons)
    mslp_minima_list_with_disturbance_threshold_booleans = calc_disturbance_threshold_booleans(mslp_minima_list_in_basins, model_name)
    # print_candidates(mslp_minima_list_with_disturbance_threshold_booleans, lats, lons, meet_all_disturbance_thresholds = True)

    updated_mslp_minima_list = []

    # Iterate over the list of candidates
    for candidate in mslp_minima_list_with_disturbance_threshold_booleans:
        # exclude candidates not meeting all criteria
        if not candidate['criteria']['all']:
            continue

        # Update a copy of the candidate's dictionary with LAT, LON
        updated_candidate = candidate.copy()
        # add the LAT, LON
        x = candidate["x_value"]
        y = candidate["y_value"]
        lat, lon = array_indices_to_lat_lon(x, y, lats, lons)
        updated_candidate['lat'] = lat
        updated_candidate['lon'] = lon
        
        # Add the updated candidate to the list
        updated_mslp_minima_list.append(updated_candidate)

    return updated_mslp_minima_list

In [4]:
# Test the function
grib_file = "navgem_2023101906f096.grib2"

#list_available_parameters(grib_file)

candidates = get_disturbance_candidates_from_unified_grib(grib_file)
print_candidates(candidates)


Converting MSLP units from Pa to hPa
# 1, IO Basin, Latitude (deg):   15.5, Longitude (deg):   53.5, MSLP (hPa):  992.7, 850 RV MAX (*10^-5 1/s): 49.59, 250-850 hPa Thickness (m): 9563.80, 925 hPa WS MAX (m/s):  57.83
# 2, IO Basin, Latitude (deg):   16.0, Longitude (deg):   84.5, MSLP (hPa):  986.1, 850 RV MAX (*10^-5 1/s): 55.36, 250-850 hPa Thickness (m): 9620.60, 925 hPa WS MAX (m/s):  58.66
# 3, AL Basin, Latitude (deg):   33.5, Longitude (deg):  -54.5, MSLP (hPa):  970.1, 850 RV MAX (*10^-5 1/s): 76.10, 250-850 hPa Thickness (m): 9682.10, 925 hPa WS MAX (m/s):  67.37


/home/db/.local/lib/python3.9/site-packages/pint/facets/plain/quantity.py:986: RuntimeWarning: invalid value encountered in divide
  magnitude = magnitude_op(new_self._magnitude, other._magnitude)
